In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import shap
from scipy import stats
import warnings
from pathlib import Path
import pickle
from datetime import datetime
import pytz
import random
import os
warnings.filterwarnings('ignore')

def keepa_to_est(keepa_time):
    """Convert Keepa time to EST datetime"""
    if isinstance(keepa_time, datetime):
        if keepa_time.tzinfo is None:
            keepa_time = pytz.UTC.localize(keepa_time)
        return keepa_time.astimezone(pytz.timezone('US/Eastern'))
    
    try:
        unix_time = (keepa_time + 21564000) * 60
        utc_time = datetime.fromtimestamp(unix_time, tz=pytz.UTC)
        return utc_time.astimezone(pytz.timezone('US/Eastern'))
    except TypeError as e:
        print(f"Error converting time: {keepa_time}, type: {type(keepa_time)}")
        raise e

def check_data_completeness(df, required_metrics):
    """
    Check if the data contains all required metrics
    Return (is_complete, missing_metrics)
    """
    existing_columns = set(df.columns)
    required_metrics = set(required_metrics)
    missing_metrics = required_metrics - existing_columns
    
    # Check if all required columns exist
    if missing_metrics:
        return False, missing_metrics
    
    # # Check if each column has enough non-null values (e.g., at least 80% of the data)
    # threshold = 0.8
    # for metric in required_metrics:
    #     if df[metric].isna().mean() > (1 - threshold):
    #         missing_metrics.add(f"{metric} (>80% missing)")
    #         return False, missing_metrics
    
    return True, set()

def process_data_to_timeseries(file_path):
    """Convert nested data structure to time series DataFrame"""
    required_metrics = {
        'SALES', 'LISTPRICE', 'BUY_BOX_SHIPPING', 
        'COUNT_REVIEWS', 'RATING'
    }
    
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        
        nested_data = data.get('data', {})
        asin = data.get('asin', 'unknown')
        
        metrics = {
            'AMAZON': ['AMAZON_time', 'AMAZON'],
            'NEW': ['NEW_time', 'NEW'],
            'USED': ['USED_time', 'USED'],
            'SALES': ['SALES_time', 'SALES'],
            'LISTPRICE': ['LISTPRICE_time', 'LISTPRICE'],
            'NEW_FBA': ['NEW_FBA_time', 'NEW_FBA'],
            'COUNT_NEW': ['COUNT_NEW_time', 'COUNT_NEW'],
            'RATING': ['RATING_time', 'RATING'],
            'COUNT_REVIEWS': ['COUNT_REVIEWS_time', 'COUNT_REVIEWS'],
            'BUY_BOX_SHIPPING': ['BUY_BOX_SHIPPING_time', 'BUY_BOX_SHIPPING']
        }
        
        time_series_dict = {}
        for metric_name, (time_key, value_key) in metrics.items():
            if time_key in nested_data and value_key in nested_data:
                times = nested_data[time_key]
                values = nested_data[value_key]
                
                if not isinstance(times, (list, np.ndarray)) or not isinstance(values, (list, np.ndarray)):
                    continue
                    
                for t, v in zip(times, values):
                    try:
                        est_time = keepa_to_est(t)
                        if est_time not in time_series_dict:
                            time_series_dict[est_time] = {}
                        time_series_dict[est_time][metric_name] = np.nan if v == -1 else v
                    except Exception as e:
                        print(f"Error processing {metric_name} for {asin}: {str(e)}")
                        continue
        
        if not time_series_dict:
            print(f"No valid data found for {asin}")
            return None, asin
        
        df = pd.DataFrame.from_dict(time_series_dict, orient='index')
        df.sort_index(inplace=True)
        
        # Check data completeness
        is_complete, missing_metrics = check_data_completeness(df, required_metrics)
        if not is_complete:
            print(f"Incomplete data for {asin}. Missing metrics: {missing_metrics}")
            return None, asin
        
        # Basic data check
        if len(df) < 60:  # Ensure at least 60 days of data
            print(f"Insufficient data points for {asin}: {len(df)} < 60 days")
            return None, asin
            
        df = df.ffill().bfill()
        df['ASIN'] = asin
        
        return df, asin
        
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None, None

def preprocess_data(df):
    """Apply all preprocessing steps"""
    # Replace inf and -inf with 0
    df = df.replace([np.inf, -np.inf], 0)

    # Calculate rolling averages
    df['Sales_rank_rolling_7'] = df['SALES'].rolling(window=7, min_periods=1).mean()
    df['Sales_rank_rolling_30'] = df['SALES'].rolling(window=30, min_periods=1).mean()
    df['Sales_rank_rolling_60'] = df['SALES'].rolling(window=60, min_periods=1).mean()
    df['Sales_rank_rolling_90'] = df['SALES'].rolling(window=90, min_periods=1).mean()
    
    df['List_price_rolling_7'] = df['LISTPRICE'].rolling(window=7, min_periods=1).mean()
    df['List_price_rolling_30'] = df['LISTPRICE'].rolling(window=30, min_periods=1).mean()
    df['Buy_box_price_rolling_7'] = df['BUY_BOX_SHIPPING'].rolling(window=7, min_periods=1).mean()
    df['Buy_box_price_rolling_30'] = df['BUY_BOX_SHIPPING'].rolling(window=30, min_periods=1).mean()
    
    # Calculate price changes (use clip to avoid -inf)
    df['List_price_change_7'] = df['LISTPRICE'].pct_change(periods=7).clip(-1, 1) * 100
    df['Buy_box_price_change_7'] = df['BUY_BOX_SHIPPING'].pct_change(periods=7).clip(-1, 1) * 100
    
    # Calculate review metrics
    df['New_reviews'] = df['COUNT_REVIEWS'].diff(periods=7).fillna(0)
    df['Review_growth_rate'] = df['COUNT_REVIEWS'].pct_change().clip(-1, 1) * 100
    
    # Calculate future metrics
    future_sales_rank = df['SALES'].shift(-30)
    current_sales_rank = df['SALES']
    df['Sales_rank_change_30d'] = ((future_sales_rank - current_sales_rank) / current_sales_rank).clip(-1, 1) * 100
    df['Sales_rank_change_30d'] = df['Sales_rank_change_30d'].fillna(0) # Ensure no NaN values
    
    # Calculate Go_viral - if the sales rank change is less than -30% (which means the sales rank is increased by more than 30%), then it is viral
    df['Go_viral'] = (df['Sales_rank_change_30d'] < -30).astype(int)

    # Check the ratio of Go_viral
    viral_ratio = df['Go_viral'].mean()
    print(f"\nViral ratio for ASIN {df['ASIN'].iloc[0]}: {viral_ratio:.2%}")

    #   Save the value of Go_viral, ensuring that subsequent processing will not change it
    go_viral_values = df['Go_viral'].copy()
    
    # If the viral ratio is less than 5%, return None, discard this ASIN
    # Because the data is not enough to train a good model
    if viral_ratio < 0.05:
        print(f"Insufficient viral ratio ({viral_ratio:.2%} < 5%) for ASIN {df['ASIN'].iloc[0]}")
        return None
    

    #  处理缺失值和异常值（除了Go_viral和ASIN）
    for col in df.columns:
        if col in ['Go_viral', 'ASIN']:  #   Skip these columns
            continue
            
        if df[col].dtype in ['float64', 'int64']:
            # For rolling and change features, use forward and backward filling
            if col.endswith(('_7', '_30', '_60', '_90', '_change_7', '_change_30d')):
                df[col] = df[col].ffill().bfill()
            else:
                # For other numeric columns, handle outliers and fill with median
                q1 = df[col].quantile(0.25)
                q3 = df[col].quantile(0.75)
                iqr = q3 - q1
                lower_bound = q1 - 1.5 * iqr
                upper_bound = q3 + 1.5 * iqr
                df[col] = df[col].clip(lower_bound, upper_bound)
                df[col] = df[col].fillna(df[col].median())

    # Rename columns for consistency
    df = df.rename(columns={
        'SALES': 'Sales_rank',
        'LISTPRICE': 'List_price',
        'BUY_BOX_SHIPPING': 'Buy_box_price',
        'COUNT_REVIEWS': 'Review_count',
        'RATING': 'Rating'
    })

    #   Restore the original value of Go_viral
    df['Go_viral'] = go_viral_values
    
    
    return df



def perform_time_series_analysis(X, y, features, max_train_ratio=0.8, random_state=42):
    """
    Use simplified time series analysis
    
    Parameters:
    -----------
    X : DataFrame
        Feature data, arranged in chronological order
    y : Series
        Target variable (Go_viral)
    features : list
        Feature name list
    max_train_ratio : float
        Training set ratio
    random_state : int
        Random seed
    """
    total_samples = len(X)
    print(f"\nTotal samples: {total_samples}")

    #### 1. Pearson Correlation Analysis ####
    # Calculate the feature correlation
    print("\nCalculating feature correlations...")
    data_for_corr = X.copy()
    data_for_corr['Go_viral'] = y
    correlation_matrix = data_for_corr.corr(method='pearson')
    
    # Create the correlation heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, 
                annot=True,
                cmap='coolwarm',
                center=0,
                fmt='.2f',
                square=True)
    plt.title('Feature Correlation Heatmap')
    plt.tight_layout()
    
    # Save the heatmap
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    heatmap_path = f'/Users/takedownccp/Documents/Cursor/DDU/data/results/correlation_heatmap_{timestamp}.png'
    plt.savefig(heatmap_path)
    plt.close()
    print(f"Correlation heatmap saved to {heatmap_path}")
    
    # Print the correlation with Go_viral
    viral_correlations = correlation_matrix['Go_viral'].sort_values(ascending=False)
    print("\nCorrelations with Go_viral:")
    print(viral_correlations)


    
    # Simple time series split
    train_size = int(total_samples * max_train_ratio)
    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
    
    print("\nDataset split:")
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Training set viral ratio: {y_train.mean():.2%}")
    print(f"Test set viral ratio: {y_test.mean():.2%}")
    
    #   Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    

    #### 2. Logistic Regression ####
    # Logistic Regression
    lr_model = LogisticRegression(
        random_state=random_state,
        class_weight='balanced',
        max_iter=1000
    )
    lr_model.fit(X_train_scaled, y_train)
    lr_pred = lr_model.predict(X_test_scaled)
    
    # Calculate LR performance metrics
    lr_metrics = {
        'accuracy': accuracy_score(y_test, lr_pred),
        'precision': precision_score(y_test, lr_pred),
        'recall': recall_score(y_test, lr_pred),
        'f1': f1_score(y_test, lr_pred)
    }
    

    #### 3. XGBoost ####
    # XGBoost
    xgb_model = xgb.XGBClassifier(
        random_state=random_state,
        scale_pos_weight=len(y_train[y_train==0])/max(1, len(y_train[y_train==1])),
        eval_metric='logloss'
    )
    
    xgb_model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )
    xgb_pred = xgb_model.predict(X_test)
    
    # Calculate XGB performance metrics
    xgb_metrics = {
        'accuracy': accuracy_score(y_test, xgb_pred),
        'precision': precision_score(y_test, xgb_pred),
        'recall': recall_score(y_test, xgb_pred),
        'f1': f1_score(y_test, xgb_pred)
    }
    
    # Feature Importance 
    lr_importance = pd.DataFrame({
        'Feature': features,
        'Coefficient': lr_model.coef_[0],
        'Abs_Coefficient': abs(lr_model.coef_[0])
    }).sort_values('Abs_Coefficient', ascending=False)
    
    xgb_importance = pd.DataFrame({
        'Feature': features,
        'Importance': xgb_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    # SHAP Analysis
    explainer = shap.TreeExplainer(xgb_model)
    shap_values = explainer.shap_values(X_test)
    
    # SHAP Analysis and Visualization
    print("\nGenerating SHAP analysis...")
    explainer = shap.TreeExplainer(xgb_model)
    shap_values = explainer.shap_values(X_test)
    
    # SHAP Summary Plot
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_test, feature_names=features, show=False)
    plt.title('SHAP Summary Plot')
    shap_path = f'/Users/takedownccp/Documents/Cursor/DDU/data/results/shap_summary_{timestamp}.png'
    plt.savefig(shap_path, bbox_inches='tight')
    plt.close()
    print(f"SHAP summary plot saved to {shap_path}")
    
    # SHAP Bar Plot
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_test, feature_names=features, plot_type='bar', show=False)
    shap_bar_path = f'/Users/takedownccp/Documents/Cursor/DDU/data/results/shap_importance_{timestamp}.png'
    plt.savefig(shap_bar_path, bbox_inches='tight')
    plt.close()
    print(f"SHAP importance plot saved to {shap_bar_path}")
    
    #  Print SHAP value statistics
    print("\nSHAP Value Statistics:")
    shap_df = pd.DataFrame(shap_values, columns=features)
    shap_importance = pd.DataFrame({
        'Feature': features,
        'Mean_SHAP': np.abs(shap_df).mean()
    }).sort_values('Mean_SHAP', ascending=False)
    print(shap_importance)
    
    # Save results
    results = {
        'correlations': viral_correlations,
        'lr_importance': lr_importance,
        'xgb_importance': xgb_importance,
        'shap_importance': shap_importance,
        'metrics': pd.DataFrame({
            'model': ['Logistic Regression', 'XGBoost'],
            'accuracy': [lr_metrics['accuracy'], xgb_metrics['accuracy']],
            'precision': [lr_metrics['precision'], xgb_metrics['precision']],
            'recall': [lr_metrics['recall'], xgb_metrics['recall']],
            'f1': [lr_metrics['f1'], xgb_metrics['f1']]
        })
    }
    
    #  Save results to CSV
    # results_path = f'/Users/takedownccp/Documents/Cursor/DDU/data/results/analysis_results_{timestamp}.csv'
    # with pd.ExcelWriter(results_path) as writer:
    #     results['correlations'].to_excel(writer, sheet_name='Correlations')
    #     results['lr_importance'].to_excel(writer, sheet_name='LR_Importance')
    #     results['xgb_importance'].to_excel(writer, sheet_name='XGB_Importance')
    #     results['shap_importance'].to_excel(writer, sheet_name='SHAP_Importance')
    #     results['metrics'].to_excel(writer, sheet_name='Model_Metrics')
    # print(f"\nAnalysis results saved to {results_path}")
    
    return (lr_importance, xgb_importance, results['metrics'], 
            xgb_model, shap_values, X_test)


def analyze_data(df, results_dir, max_attempts=10):
    """Perform all analyses with retry mechanism"""
    for attempt in range(max_attempts):
        try:
            print(f"\nAnalysis attempt {attempt + 1}/{max_attempts}")
            
            # 1. Correlation Analysis
            numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
            correlations = df[numerical_cols].corr()['Go_viral'].sort_values(ascending=False)
            
            # 2. Prepare features
            features = ['Sales_rank', 'List_price', 'Buy_box_price', 'Review_count', 'Rating',
                       'Sales_rank_rolling_7', 'Sales_rank_rolling_30', 'Sales_rank_rolling_60', 
                       'Sales_rank_rolling_90','List_price_rolling_7', 'List_price_rolling_30',
                       'Buy_box_price_rolling_7','Buy_box_price_rolling_30', 'List_price_change_7',
                       'Buy_box_price_change_7', 'New_reviews', 'Review_growth_rate']
            
            X = df[features]
            y = df['Go_viral']
            
            #  3. Execute time series analysis - use the correct parameters
            lr_importance, xgb_importance, metrics_df, final_model, shap_values, final_test_data = perform_time_series_analysis(
                X=X,
                y=y,
                features=features,
                # min_positive_samples=10,  # Ensure at least 10 positive samples
                max_train_ratio=0.8,      # Maximum training set ratio
                random_state=42+attempt   # Use different random seeds for each attempt
            )
            
            #  Save model metrics
            metrics_path = Path(results_dir) / f'model_metrics_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            metrics_df.to_csv(metrics_path, index=False)
            print(f"\nModel metrics saved to {metrics_path}")
            
            print("\nAnalysis completed successfully!")
            return correlations, lr_importance, xgb_importance, final_model
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {str(e)}")
            if attempt == max_attempts - 1:
                raise Exception(f"Analysis failed after {max_attempts} attempts")
            continue


def get_project_paths():
    """Get all project-related paths"""
    # Use the current working directory in Jupyter Notebook
    current_dir = Path.cwd()
    
    # If the current directory is 'src', then the parent directory is the project root
    if current_dir.name == 'src':
        project_root = current_dir.parent
    else:
        # Otherwise, assume the current directory is the project root
        project_root = current_dir
    
     # Construct all necessary paths
    paths = {
        'root': project_root,
        'raw_data': project_root / 'data' / 'raw_data',
        'processed_data': project_root / 'data' / 'processed_data',
        'results': project_root / 'results'
    }
    
    # Ensure that the required directories exist
    paths['processed_data'].mkdir(parents=True, exist_ok=True)
    paths['results'].mkdir(parents=True, exist_ok=True)
    
    # Verify that the raw_data directory exists
    if not paths['raw_data'].exists():
        raise FileNotFoundError(f"Raw data directory not found at {paths['raw_data']}")
        
    # Print the current path information (for debugging)
    print(f"Current directory: {current_dir}")
    print(f"Project root: {project_root}")
    print(f"Raw data path: {paths['raw_data']}")
    
    return paths




def main():
    """Main execution function"""
    # Get project paths
    try:
        paths = get_project_paths()
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return
    
    # Get timestamp (all files share the same timestamp)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Get pkl file list
    pkl_files = list(paths['raw_data'].glob('*_raw.pkl'))
    if not pkl_files:
        print(f"No pkl files found in {paths['raw_data']}")
        return
        
    print(f"Found {len(pkl_files)} pkl files in {paths['raw_data']}")
    
    successful_files = []
    valid_dfs = []
    processed_asins = set()
    max_attempts = len(pkl_files)
    attempts = 0
    
    while len(valid_dfs) <= 100 and attempts < max_attempts:
        remaining_files = [f for f in pkl_files if f.stem.split('_')[0] not in processed_asins]
        if not remaining_files:
            break
            
        file_path = random.choice(remaining_files)
        asin = file_path.stem.split('_')[0]
        processed_asins.add(asin)
        
        print(f"\nProcessing {file_path.name}...")
        try:
            df, current_asin = process_data_to_timeseries(file_path)
            if df is not None:
                df = preprocess_data(df)
                if df is not None:  # viral比例足够
                    valid_dfs.append(df)
                    successful_files.append(file_path)  # 保存成功的文件路径
                    print(f"Successfully processed {current_asin}, viral ratio: {df['Go_viral'].mean():.2%}")
                else:
                    print(f"Skipped {asin} due to insufficient viral ratio")
            else:
                print(f"Skipped {asin} due to invalid or incomplete data")
        except Exception as e:
            print(f"Error processing {asin}: {str(e)}")
        
        attempts += 1
        print(f"Progress: {len(valid_dfs)}/100 valid files found ({attempts}/{max_attempts} attempts)")
    
    if not valid_dfs:
        print("No valid data files found after processing")
        return
    
    
    # Merge all dataframes
    final_df = pd.concat(valid_dfs, ignore_index=True)

    viral_ratio = final_df['Go_viral'].mean()
    print(f"\nViral ratio for final_df: {viral_ratio:.2%}")

    
    # Save processed data
    output_path = paths['processed_data'] / f'processed_timeseries_{timestamp}.csv'
    final_df.to_csv(output_path, index=False)
    print(f"\nProcessed data saved to {output_path}")
    
    # Analyze data
    try:
        print("\nStarting data analysis...")
        correlations, lr_importance, xgb_importance, xgb_model = analyze_data(final_df, paths['results'])
        
        # Print results
        print("\n=== Analysis Results ===")
        print("\nTop 10 Correlations with Go_viral:")
        print(correlations.head(10))
        
        print("\nTop 10 Important Features (Logistic Regression):")
        print(lr_importance.head(10))
        
        print("\nTop 10 Important Features (XGBoost):")
        print(xgb_importance.head(10))
        
        # Save analysis results
        results_path = paths['results'] / f'analysis_results_{timestamp}.csv'
        results_df = pd.DataFrame({
            'Feature': lr_importance['Feature'],
            'LR_Coefficient': lr_importance['Coefficient'],
            'XGB_Importance': xgb_importance['Importance']
        })
        results_df.to_csv(results_path, index=False)
        print(f"\nAnalysis results saved to {results_path}")
        
    except Exception as e:
        print(f"\nError during analysis: {str(e)}")
        print("Analysis results not saved due to error")
    
    # Print summary
    print("\n=== Processing Summary ===")
    print(f"Total files attempted: {attempts}")
    print(f"Successfully processed files: {len(valid_dfs)}")
    print(f"Files with sufficient viral ratio: {len(valid_dfs)}")
    print(f"Timestamp: {timestamp}")
    print("\nOutput files:")
    print(f"- Processed data: processed_timeseries_{timestamp}.csv")
    print(f"- Meta information: selected_asins_meta_{timestamp}.csv")
    print(f"- Analysis results: analysis_results_{timestamp}.csv")
    print(f"- Correlation heatmap: correlation_heatmap_{timestamp}.png")
    print(f"- SHAP importance plot: shap_importance_{timestamp}.png")


if __name__ == "__main__":
    main()

Current directory: /Users/takedownccp/Documents/Cursor/DDU/src
Project root: /Users/takedownccp/Documents/Cursor/DDU
Raw data path: /Users/takedownccp/Documents/Cursor/DDU/data/raw_data
Found 909 pkl files in /Users/takedownccp/Documents/Cursor/DDU/data/raw_data

Processing B08L7LD72W_raw.pkl...

Viral ratio for ASIN B08L7LD72W: 8.03%
Successfully processed B08L7LD72W, viral ratio: 8.03%
Progress: 1/100 valid files found (1/909 attempts)

Processing B07D562VQ6_raw.pkl...

Viral ratio for ASIN B07D562VQ6: 8.73%
Successfully processed B07D562VQ6, viral ratio: 8.73%
Progress: 2/100 valid files found (2/909 attempts)

Processing B000PKWKMQ_raw.pkl...

Viral ratio for ASIN B000PKWKMQ: 7.91%
Successfully processed B000PKWKMQ, viral ratio: 7.91%
Progress: 3/100 valid files found (3/909 attempts)

Processing B01MRLPOK6_raw.pkl...

Viral ratio for ASIN B01MRLPOK6: 8.53%
Successfully processed B01MRLPOK6, viral ratio: 8.53%
Progress: 4/100 valid files found (4/909 attempts)

Processing B08HPHF91